# Bayesian estimation of open loop TAR models


In [1]:
import ipykernel
import numpy as np
import TAR
import pandas as pd
from scipy.special import gamma, factorial
from scipy.stats import bernoulli
import matplotlib.pyplot as plt
import random
import numbers

c:\Users\samir\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
c:\Users\samir\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [81]:

# Generate TAR series:
y, s = TAR.tar_simulation([2,-0.9], [-2,0.5], 30, sd = 0.5, tau = 0)
dat = pd.DataFrame({"y":[i[0] for i in y], "s_1":s})
dat['x'] = dat["y"].shift(1)
dat.dropna(inplace = True)
dat.drop("s_1", axis=1, inplace = True)
# Some random partition:
dat["s_1"] = [bernoulli.rvs(0.5) for i in range(len(dat))]
orig = dat.copy()

{'Regime_1': 14, 'Regime_2': 16}


In [62]:
def DataProcessing(dat_init, first = True):

    data = dat_init.drop(dat.columns[3:], axis = 1)
    
    n                   =       len(data
    )
    n_1                 =       np.sum(data.s_1)
    data["xs_1"]        =       data.x*data.s_1 
    data["s_2"]         =       1 - data.s_1
    data["xs_2"]        =       data["s_2"]*data["x"
    ]
    data["ys_1"]        =       data.y*data.s_1
    data["ys_2"]        =       data.y*data.s_2

##############################################################################################################################################
############################################################### Covariances ##################################################################
##############################################################################################################################################
    if first == True:
        Sxx_1               =       (data.loc[:, ["s_1", "xs_1"]].T@data.loc[:, ["s_1", "xs_1"           ]
        ]).to_numpy()       # shape(2,2)
        assert(Sxx_1.shape == (2,2))
        Sxx_2               =       (data.loc[:, ["s_2", "xs_2"]].T@data.loc[:, ["s_2", "xs_2"           ]
        ]).to_numpy()       # shape(2,2)
        assert(Sxx_2.shape == (2,2))
        Sxy_1               =       (data.loc[:, ["s_1", "xs_1"]].T@data.loc[:, ["ys_1"                  ]          
        ]).to_numpy()       # shape(2,2)
        assert(Sxy_1.shape == (2,1))
        Sxy_2               =       (data.loc[:, ["s_2", "xs_2"]].T@data.loc[:, ["ys_2"                  ]          
        ]).to_numpy()       # shape(2,1)
        assert(Sxy_2.shape == (2,1))
        Syy_1               =       (data.loc[:, "ys_1"].T@data.loc[:, "ys_1"                            ])
        assert(isinstance(Syy_1, numbers.Number)==1)
        Syy_2               =       (data.loc[:, "ys_2"].T@data.loc[:, "ys_2"                            ])
        assert(isinstance(Syy_2, numbers.Number)==1)
        inv_1               =       np.linalg.inv(Sxx_1)
        assert(inv_1.shape == (2,2))
        inv_2               =       np.linalg.inv(Sxx_2)
        assert(inv_2.shape == (2,2))
        det_1               =       np.linalg.det(Sxx_1)
        det_2               =       np.linalg.det(Sxx_2)
        assert(isinstance(det_1, numbers.Number)==1 and isinstance(det_2, numbers.Number)==1)
        dictionary = {"inverses": [inv_1, inv_2], "determinants":[det_1, det_2], "covs":[Sxx_1, Sxx_2, Sxy_1, Sxy_2, Syy_1, Syy_2]}
        return data, dictionary
    else:
        return data
    
##############################################################################################################################################
############################################################### Recursions ###################################################################
##############################################################################################################################################

def Recursions(dictionary, xi, yi, omega, n_1):
    # Unpacking:
    inv_1, inv_2 = dictionary["inverses"]
    det_1, det_2 = dictionary["determinants"]
    Sxx_1, Sxx_2, Sxy_1, Sxy_2, Syy_1, Syy_2 = dictionary["covs"]
    # Recursions:
    inv1 = inv_1 - inv_1@xi@xi.T@inv_1/(xi.T@inv_1@xi + (-1)**(omega))
    inv2 = inv_2 - inv_2@xi@xi.T@inv_2/(xi.T@inv_2@xi - (-1)**(omega))
    det1 = abs(det_1*(xi.T@inv_1@xi + (-1)**(omega)))[0][0]
    det2 = abs(det_2*(xi.T@inv_2@xi - (-1)**(omega)))[0][0]
    Sxx1 = Sxx_1 + (-1)**(omega)*xi@xi.T
    Sxx2 = Sxx_2 - (-1)**(omega)*xi@xi.T
    Syy1 = Syy_1 + (-1)**(omega)*yi*yi
    Syy2 = Syy_2 - (-1)**(omega)*yi*yi
    Sxy1 = Sxy_1 + (-1)**(omega)*xi*yi
    Sxy2 = Sxy_2 - (-1)**(omega)*xi*yi
    n_1 = n_1 + (-1)**omega
    # assert(inv_1.shape == (2,2) and inv_2.shape == (2,2) and Sxx_1.shape == (2,2) and Sxx_2.shape == (2,2))
    dictionary = {"inverses": [inv1, inv2], "determinants":[det1, det2], "covs":[Sxx1, Sxx2, Sxy1, Sxy2, Syy1, Syy2], "n_1": n_1}
    return dictionary

def Quantity(n, n_1, delta_1, delta_2, det):
    return np.log(gamma((n_1 +0.01)/2                                          )
    ) + np.log(gamma((n - n_1 + 0.01)/2                                                             )
    ) - np.log(delta_1**((n_1 - 1 -1)/2))  - np.log(delta_2**((n - n_1 - 1 -1)/2                    )
    ) - np.log(0.01 + det**0.5                                                          
    )

    
        

In [89]:
def Update(data, dictionary = None, row = 1, first = True):
    if first == True:
        data, dictionary = DataProcessing(data, first = True)
        return data, dictionary
    else:
        dicts = {}
        data = DataProcessing(data, first = False)
        omega = (data.loc[row, "s_1"]==1).astype(int)
        n, n_1 = len(data), np.sum(data.loc[:, "s_1"])
        xi, yi = data.loc[row,[f"s_{2-omega}", f"xs_{2-omega}"]].to_numpy().reshape(-1,1), data.loc[row, "y"]
        dicts[f'dictionary_{2-omega}'] = dictionary
        inv_1, inv_2 = dictionary["inverses"]
        det_1, det_2 = dictionary["determinants"]
        Sxx_1, Sxx_2, Sxy_1, Sxy_2, Syy_1, Syy_2 = dictionary["covs"]
        delta_1             =       (data.ys_1.T@data.ys_1 - Sxy_1.T@inv_1@Sxy_1)[0][0]
        delta_2             =       (data.ys_2.T@data.ys_2 - Sxy_2.T@inv_2@Sxy_2)[0][0]
        det = det_1*det_2
        Q = {}
        Q[f'Q_{2-omega}'] = Quantity(n, n_1, delta_1, delta_2, det)

        dictionary = Recursions(dictionary, xi, yi, omega, n_1)
        dicts[f'dictionary_{2-(1-omega)}'] = dictionary
        inv_1, inv_2 = dictionary["inverses"]
        det_1, det_2 = dictionary["determinants"]
        Sxx_1, Sxx_2, Sxy_1, Sxy_2, Syy_1, Syy_2 = dictionary["covs"]
        n_1 = dictionary["n_1"]
        delta_1             =       (Syy_1 - Sxy_1.T@inv_1@Sxy_1)[0][0]
        delta_2             =       (Syy_2 - Sxy_2.T@inv_2@Sxy_2)[0][0]
        det = det_1*det_2
        omega = 1 - omega
        Q[f'Q_{2-omega}'] = omega*Quantity(n, n_1, delta_1, delta_2, det) + (1-omega)*Quantity(n, n_1, delta_1, delta_2, det)
        C                    =       max(Q["Q_1"], Q["Q_2"]
        )
        Q_1                 =       Q["Q_1"] - C
        Q_2                 =  Q["Q_2"] - C
        p                    =       np.exp(Q_1)/(np.exp(Q_1) + np.exp(Q_2                              )
        )
        data.loc[row, 
            "s_1"]               =       bernoulli.rvs(p
            )
        if data.loc[row, "s_1"] == 1:
            dictionary = dicts[f'dictionary_{1}']
        else:
            dictionary = dicts[f'dictionary_{2}']
        return data, dictionary, p


In [90]:
probs = pd.DataFrame()
frames = []
ps = []
N = 1000
rows = dat.index.to_list()
for j in range(N):
    if j == 0:
        data, dictionary = Update(dat, first = True)
    else:
        frames += [pd.Series(ps, index = rows, name = f"i{j}")]
        ps = []
    random.shuffle(rows)
    for row in rows:
        data, dictionary, p = Update(data, dictionary=dictionary, row = row, first = False)
        ps.append(p)
        if j == N-1 and row == rows[-1]:
            frames += [pd.Series(ps, index = rows, name = f"i{j+1}")]
            probs = pd.concat(frames, axis=1)

In [91]:
probs.loc[:, "state"] = s[1:]
probs

,i1,i2,i3,i4,i5,i6,i7,i8,i9,i10,...,i992,i993,i994,i995,i996,i997,i998,i999,i1000,state
7,1.000000,1.000000,1.000000,0.999977,9.999959e-01,9.999727e-01,9.999893e-01,9.999960e-01,9.999545e-01,9.999988e-01,...,1.000000,1.000000,9.999979e-01,0.999999,0.999999,0.999999,0.999989,0.999970,9.999994e-01,1
18,0.399326,0.589859,0.672187,0.184170,2.544690e-01,2.957949e-01,3.517948e-01,3.517948e-01,3.311543e-01,5.018373e-01,...,0.337685,0.534250,2.347996e-01,0.148875,0.084283,0.084283,0.192640,0.344285,1.660599e-01,0
19,0.999997,0.999981,0.999999,0.999980,9.996440e-01,9.997367e-01,9.999384e-01,9.998944e-01,9.998944e-01,9.999655e-01,...,0.999992,0.999995,9.999605e-01,0.999992,0.999953,0.999953,0.999981,0.999705,9.999879e-01,0
24,0.463179,0.573127,0.555120,0.273285,3.004069e-01,5.581151e-01,4.904549e-01,5.614130e-01,5.296490e-01,6.250903e-01,...,0.584627,0.581497,3.846757e-01,0.334646,0.178180,0.178180,0.264728,0.474204,2.649736e-01,1
2,0.010951,0.001476,0.002876,0.000005,3.701665e-06,2.048227e-05,3.214514e-05,7.907600e-05,2.386468e-05,4.658100e-05,...,0.000923,0.001052,1.725502e-05,0.000060,0.000009,0.000009,0.000029,0.000027,1.889021e-04,0
27,0.999998,0.999983,0.999999,0.999933,9.999687e-01,9.998266e-01,9.999218e-01,9.999658e-01,9.997951e-01,9.999779e-01,...,0.999986,0.999996,9.999821e-01,0.999993,0.999993,0.999984,0.999926,0.999986,9.999971e-01,0
6,0.000023,0.000001,0.000030,0.000002,3.706650e-07,3.192536e-07,2.205755e-07,5.880892e-07,4.194109e-07,7.220593e-07,...,0.000003,0.000006,7.447106e-07,0.000004,0.000003,0.000001,0.000001,0.000001,9.092686e-07,1
5,0.261415,0.492089,0.265508,0.005425,6.712849e-03,3.168698e-02,1.532694e-02,5.486091e-02,6.252594e-02,1.374715e-01,...,0.272865,0.075558,7.178013e-03,0.014716,0.002269,0.003808,0.008785,0.012233,2.327604e-01,0
12,0.182947,0.286843,0.286843,0.518280,5.681292e-01,5.598783e-01,5.928694e-01,5.904384e-01,5.579857e-01,5.131109e-01,...,0.487797,0.520544,5.325903e-01,0.406322,0.358563,0.411194,0.501100,0.592250,3.561736e-01,1
10,0.091312,0.153567,0.078340,0.291489,3.461781e-01,1.469378e-01,2.539789e-01,2.537113e-01,3.127119e-01,2.345512e-01,...,0.146900,0.301911,4.576164e-01,0.537865,0.541768,0.541768,0.435565,0.298030,3.683591e-01,0


In [84]:
rows = dat.index.to_list()
random.shuffle(rows)
for j in range(1000):
    if j == 0:
        data, dictionary = Update(dat, first = True)
    else:
        for row in rows:
            data, dictionary = Update(data, dictionary, row = row, first = False)

1.2690675225280188
Q = {'Q_1': -3.417931798676176, 'Q_2': -18.04040255263265}
0.999999553787794
4.3143714539721145
Q = {'Q_2': -3.417931798676176, 'Q_1': -7.921249196399671}
0.010950953441346323
11.246458171919272
Q = {'Q_2': -3.417931798676176, 'Q_1': -13.487987882621358}
4.2326448329385424e-05
2.1470074833178785
Q = {'Q_1': -3.417931798676176, 'Q_2': -18.261739792634813}
0.9999996423848223
3.1134823707439523
Q = {'Q_2': -3.417931798676176, 'Q_1': -4.4565613968515665}
0.2614145004092719
23.552244140828762
Q = {'Q_2': -4.456561396851626, 'Q_1': -15.542916200358246}
1.5319713063909816e-05
3.0907948893694197
Q = {'Q_1': -4.456561396851626, 'Q_2': -20.70601781503833}
0.9999999123098716
3.409491613116046
Q = {'Q_2': -4.456561396851626, 'Q_1': -5.374139245758286}
0.2854516812342804
3.0179825251621253
Q = {'Q_1': -4.456561396851626, 'Q_2': -17.572440153134593}
0.9999979869926686
5.1800598613270665
Q = {'Q_2': -4.456561396851626, 'Q_1': -7.875061268396022}
0.03172227392702938
3.10174111679288